In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import datetime

# ==========================================
# 1. PRICING MODELS
# ==========================================

def black_scholes_fx(S, K, T, r_dom, r_for, sigma, option_type='call'):
    """
    Garman-Kohlhagen model for FX options.
    S: Spot FX Rate (BRL/USD)
    K: Strike Price
    T: Time to Maturity (years)
    r_dom: Domestic Risk-Free Rate (BRL)
    r_for: Foreign Risk-Free Rate (USD)
    sigma: Volatility
    """
    if T <= 0:
        return max(0, S - K) if option_type == 'call' else max(0, K - S)

    d1 = (np.log(S / K) + (r_dom - r_for + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        price = S * np.exp(-r_for * T) * norm.cdf(d1) - K * np.exp(-r_dom * T) * norm.cdf(d2)
    else:
        price = K * np.exp(-r_dom * T) * norm.cdf(-d2) - S * np.exp(-r_for * T) * norm.cdf(-d1)

    return price

def price_future_mtm(current_price, delivery_price, contract_size):
    """
    Calculates the value of a Futures position relative to the delivery price.
    Note: Futures are daily settled, but for portfolio value we look at
    accumulated gain/loss vs the delivery price.
    """
    return (current_price - delivery_price) * contract_size

def price_bond(notional, coupon_rate, T, r_yield, freq=2):
    """
    Prices a bullet bond.
    coupon_rate: Annual coupon rate
    freq: Coupons per year (2 = semi-annual)
    r_yield: Current market yield for similar maturity (annual)
    """
    if T <= 0: return 0

    periods = int(np.ceil(T * freq))
    coupon_payment = notional * (coupon_rate / freq)
    dt = 1 / freq

    pv = 0
    # Discount coupons
    for i in range(1, periods + 1):
        t_flow = i * dt
        # Using exponential discounting for simplicity in calculation
        pv += coupon_payment * np.exp(-r_yield * t_flow)

    # Discount principal
    t_final = periods * dt
    pv += notional * np.exp(-r_yield * t_final)

    return pv

# ==========================================
# 2. MARKET DATA & SETUP
# ==========================================

# Assumptions for "Current" Market State (Nov 21, 2024)
# Used as a baseline to calculate the P&L impacts
market_data = {
    'date': datetime.date(2024, 11, 21),
    'USD_BRL': 5.80,            # Current Spot
    'IBOVESPA': 128000,         # Current Index Points
    'r_brl': 0.1275,            # ~12.75% (Selic/DI)
    'r_usd': 0.045,             # ~4.5% (US Treasuries)
    'sigma_usd': 0.15,          # 15% Volatility
    'bond_yield_1y': 0.13,      # Market yield for 1y corporate risk
    'bond_yield_3y': 0.135      # Market yield for 3y corporate risk
}

# Portfolio Specifications
positions = {
    'A_Calls': {
        'type': 'option_call',
        'underlying': 'USD',
        'strike': 5.35,         # 5350 points = 5.35 BRL
        'maturity': datetime.date(2025, 12, 1),
        'qty': 300,
        'multiplier': 50000     # Standard B3 USD Option size is $50k
    },
    'B_Futures': {
        'type': 'future',
        'underlying': 'IBOV',
        'delivery_price': 165000,
        'maturity': datetime.date(2026, 2, 1),
        'qty': 250,
        'multiplier': 1.0       # B3 Standard: R$1 per point
    },
    'C_Bond_1y': {
        'type': 'bond',
        'notional': 1000,
        'coupon': 0.06,         # 6% p.a.
        'maturity_years': 1.0,
        'qty': 25000
    },
    'D_Bond_3y': {
        'type': 'bond',
        'notional': 1000,
        'coupon': 0.064,        # 6.4% p.a.
        'maturity_years': 3.0,
        'qty': 25000
    }
}

def calculate_portfolio_value(mkt, pos):
    """
    Calculates Total Portfolio Value (BRL) given a market state dictionary.
    """
    # Time to maturity calculations
    today = mkt['date']

    # 1. Option Position
    # T in years
    t_opt = (pos['A_Calls']['maturity'] - today).days / 365.0
    unit_price_opt = black_scholes_fx(
        S=mkt['USD_BRL'],
        K=pos['A_Calls']['strike'],
        T=t_opt,
        r_dom=mkt['r_brl'],
        r_for=mkt['r_usd'],
        sigma=mkt['sigma_usd']
    )
    # Value = Price * Qty * Multiplier (50k USD)
    val_opt = unit_price_opt * pos['A_Calls']['qty'] * pos['A_Calls']['multiplier']

    # 2. Futures Position
    # Value = (Current - Delivery) * Qty * Multiplier
    val_fut = price_future_mtm(
        current_price=mkt['IBOVESPA'],
        delivery_price=pos['B_Futures']['delivery_price'],
        contract_size=pos['B_Futures']['multiplier']
    ) * pos['B_Futures']['qty']

    # 3. Bond 1Y
    # Assume yield curve shifts parallel with r_brl changes relative to baseline
    spread_1y = mkt.get('bond_yield_1y', 0.13) - 0.1275 # spread over risk free
    curr_yield_1y = mkt['r_brl'] + spread_1y

    unit_price_bond1 = price_bond(
        notional=pos['C_Bond_1y']['notional'],
        coupon_rate=pos['C_Bond_1y']['coupon'],
        T=pos['C_Bond_1y']['maturity_years'],
        r_yield=curr_yield_1y
    )
    val_bond1 = unit_price_bond1 * pos['C_Bond_1y']['qty']

    # 4. Bond 3Y
    spread_3y = mkt.get('bond_yield_3y', 0.135) - 0.1275
    curr_yield_3y = mkt['r_brl'] + spread_3y

    unit_price_bond3 = price_bond(
        notional=pos['D_Bond_3y']['notional'],
        coupon_rate=pos['D_Bond_3y']['coupon'],
        T=pos['D_Bond_3y']['maturity_years'],
        r_yield=curr_yield_3y
    )
    val_bond3 = unit_price_bond3 * pos['D_Bond_3y']['qty']

    total_value = val_opt + val_fut + val_bond1 + val_bond3

    return {
        'Total': total_value,
        'Option_Val': val_opt,
        'Future_Val': val_fut,
        'Bond1y_Val': val_bond1,
        'Bond3y_Val': val_bond3
    }

# ==========================================
# 3. EXECUTE SCENARIOS
# ==========================================

# --- Question 1: Current Value ---
current_results = calculate_portfolio_value(market_data, positions)

# --- Question 2: Defined Shock Scenario ---
# USD-BRL, Ibovespa, Volatility all UP 10%
# Risk-free spot rate curve parallel shift 100 bps UP and DOWN

scenario_2_up = market_data.copy()
scenario_2_up['USD_BRL'] *= 1.10
scenario_2_up['IBOVESPA'] *= 1.10
scenario_2_up['sigma_usd'] *= 1.10
scenario_2_up['r_brl'] += 0.01 # +100 bps

scenario_2_down = market_data.copy()
scenario_2_down['USD_BRL'] *= 1.10
scenario_2_down['IBOVESPA'] *= 1.10
scenario_2_down['sigma_usd'] *= 1.10
scenario_2_down['r_brl'] -= 0.01 # -100 bps

res_2_up = calculate_portfolio_value(scenario_2_up, positions)
res_2_down = calculate_portfolio_value(scenario_2_down, positions)

pnl_2_up = res_2_up['Total'] - current_results['Total']
pnl_2_down = res_2_down['Total'] - current_results['Total']

# --- Question 3: 2008 Crisis Stress Test ---
# "Worst market movements between Jan/2007 and Dec/2009"
# The 2008 crisis is the key event.
# Characteristics:
# 1. USD Exploded (Flight to safety). From ~1.60 to ~2.50 (+56%)
# 2. Ibovespa Crashed. From ~73k to ~29k (-60%)
# 3. Rates Spiked.
# We will apply a conservative "Crisis" shock:
# USD: +40%, Ibovespa: -40%, Rates: +300bps (3%), Vol: +50%

scenario_2008 = market_data.copy()
scenario_2008['USD_BRL'] *= 1.40   # Massive depreciation of BRL
scenario_2008['IBOVESPA'] *= 0.60  # Market crash (-40%)
scenario_2008['sigma_usd'] *= 1.50 # Volatility spike
scenario_2008['r_brl'] += 0.03     # Rate hike crisis defense

res_2008 = calculate_portfolio_value(scenario_2008, positions)
pnl_2008 = res_2008['Total'] - current_results['Total']

# ==========================================
# 4. PRINT REPORT
# ==========================================

def format_currency(val):
    return f"R$ {val:,.2f}"

print(f"{'='*60}")
print(f"RISK MANAGEMENT REPORT")
print(f"{'='*60}")

print(f"\n1. CURRENT PORTFOLIO VALUE")
print(f"-"*30)
print(f"Long USD Calls:      {format_currency(current_results['Option_Val'])}")
print(f"Long Ibovespa Fut:   {format_currency(current_results['Future_Val'])}")
print(f"Corp Bond (1y):      {format_currency(current_results['Bond1y_Val'])}")
print(f"Corp Bond (3y):      {format_currency(current_results['Bond3y_Val'])}")
print(f"------------------------------")
print(f"TOTAL VALUE:         {format_currency(current_results['Total'])}")

print(f"\n\n2. SENSITIVITY ANALYSIS (Daily P&L Impact)")
print(f"(Scenario: USD+10%, Ibov+10%, Vol+10%)")
print(f"-"*30)
print(f"With Rates +100bps:  {format_currency(pnl_2_up)}")
print(f"With Rates -100bps:  {format_currency(pnl_2_down)}")
print(f"\nAnalysis: The portfolio benefits from USD and Equity rising.")
print(f"However, rising rates (+100bps) hurt the bond prices significantly.")

print(f"\n\n3. STRESS TEST (Historical: 2007-2009 Crisis)")
print(f"Scenario: USD +40%, Ibov -40%, Rates +300bps, Vol +50%")
print(f"-"*30)
print(f"Stress P&L:          {format_currency(pnl_2008)}")
print(f"\nBreakdown of Stress P&L:")
print(f"USD Call Gain:       {format_currency(res_2008['Option_Val'] - current_results['Option_Val'])}")
print(f"Ibov Future Loss:    {format_currency(res_2008['Future_Val'] - current_results['Future_Val'])}")
print(f"Bond Portfolio Loss: {format_currency((res_2008['Bond1y_Val'] + res_2008['Bond3y_Val']) - (current_results['Bond1y_Val'] + current_results['Bond3y_Val']))}")
print(f"{'='*60}")

RISK MANAGEMENT REPORT

1. CURRENT PORTFOLIO VALUE
------------------------------
Long USD Calls:      R$ 13,486,422.55
Long Ibovespa Fut:   R$ -9,250,000.00
Corp Bond (1y):      R$ 23,313,757.94
Corp Bond (3y):      R$ 20,489,650.99
------------------------------
TOTAL VALUE:         R$ 48,039,831.48


2. SENSITIVITY ANALYSIS (Daily P&L Impact)
(Scenario: USD+10%, Ibov+10%, Vol+10%)
------------------------------
With Rates +100bps:  R$ 10,922,857.76
With Rates -100bps:  R$ 11,163,684.59

Analysis: The portfolio benefits from USD and Equity rising.
However, rising rates (+100bps) hurt the bond prices significantly.


3. STRESS TEST (Historical: 2007-2009 Crisis)
Scenario: USD +40%, Ibov -40%, Rates +300bps, Vol +50%
------------------------------
Stress P&L:          R$ 19,519,441.65

Breakdown of Stress P&L:
USD Call Gain:       R$ 34,616,583.51
Ibov Future Loss:    R$ -12,800,000.00
Bond Portfolio Loss: R$ -2,297,141.86
